In [1]:
from openvino.runtime import Core
import numpy as np

core = Core()

model = core.read_model('yolo_v5s.xml')
compiled_model = core.compile_model(model, 'CPU')
infer_request = compiled_model.create_infer_request()

In [2]:
from openvino.model_zoo.datasets import COCO2017Dataset
from openvino.model_zoo.transforms.input import TransformComposition, pillow_resize, transpose
from openvino.model_zoo.transforms.output import YoloV5
from openvino.model_zoo.metrics import create_coco_ap

In [3]:

# create preprocessing outsize dataset
preprocessor = TransformComposition([
    pillow_resize(size=640), 
    transpose([2, 0, 1]),
]
)

coco_dataset = COCO2017Dataset(annotation_file='MSCOCO/annotations/instances_val2017.json', data_dir='MSCOCO/val2017')
postprocessor = YoloV5(compiled_model.outputs)
metric = create_coco_ap(dataset=coco_dataset)

Annotation conversion for coco2017 dataset has been started
Parameters to be used for conversion:
converter: mscoco_detection
annotation_file: MSCOCO/annotations/instances_val2017.json
Annotation conversion for coco2017 dataset has been finished


TypeError: __init__() got multiple values for argument 'dataset'

In [ ]:
for (batch_input_ids, batch_annotation, batch_input, batch_meta, batch_identifiers) in coco_dataset:
    batch_input, batch_meta = preprocessor(batch_input, batch_meta)
    infer_result = infer_request.infer(np.array(batch_input))
    batch_predictions = postprocessor(infer_result, batch_meta, batch_identifiers)
    metric.batch_update(batch_annotation, batch_predictions)

NameError: name 'coco_dataset' is not defined

In [ ]:
from openvino.inference_engine import IECore
from openvino.model_zoo.model_api.models import SSD
from openvino.model_zoo.model_api.adapters import OpenvinoAdapter

ssd_model = SSD(OpenvinoAdapter(IECore(), 'ssdlite_mobilenet_v2.xml'))
for (_, _, batch_input, _, _) in coco_dataset[:10]:
    output = ssd_model(batch_input[0])


In [ ]:
from openvino.model_zoo.model_api.models import DetectionModel
from openvino.model_zoo.model_api.models.utils import Detection

class YoloV5Model(DetectionModel):
    def __init__(self, model_adapter, configuration=None, preload=False):
        super().__init__(model_adapter, configuration, preload)
        self._preprocessor = TransformComposition(pillow_resize(size=640), transpose([2, 0, 1]))
        self._postprocessor = YoloV5(model_adapter.outputs)

    def preprocess(self, inputs):
        inputs, meta = self._preprocessor([inputs])
        dict_inputs = {self.image_blob_name: np.array(inputs)}
        return dict_inputs, meta[0]

    def postprocess(self, outputs, meta):
        detection = self._postprocessor(outputs, meta)[0]
        list_boxes = [
            Detection(xmin, ymin, xmax, ymax, score, lbl) 
            for xmin, ymin, xmax, ymax, score, lbl in zip(
            detection.x_mins, 
            detection.y_mins, 
            detection.x_maxs, 
            detection.y_maxs, 
            detection.scores, 
            detection.labels
            )]

        return list_boxes